# Constraining the A.v. model main carbon metabolism 

There are particular constraints of A.v. that are currently not present in the model such as the useage of the Entner–Doudoroff Pathway (ED) and the pentose phosphate pathway (PPP) as well as sucrose metabolism particularly fructose catabolism. 

While we have MFA results (Wu et al. 2019) to constrain the model but we would like to use that data to test the model so we will try to constrain the pathways with gene knockout first. 





